<a href="https://colab.research.google.com/github/napoles-uach/streamlit_apps/blob/main/Streamlit_Colab/06_Streamlit__Colab_BrainTumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 6. Brain Tumor MRI Classification streamlit app. 
## This notebook shows how to deploy an Streamlit app on top of Google Colaboratory (GPU's) using a trained model with Google's [Teachable Machine](https://teachablemachine.withgoogle.com/train).

![](https://github.com/napoles-uach/streamlit_apps/blob/main/Streamlit_Colab/Images/06.png?raw=True)

## 1) Run all and click the "**Link to web app**" at the bottom.
## 2) Modify the code in `app.py`


# Setup
(pip install what you need ...) 

In [ ]:
pip install keras streamlit pillow numpy

In [ ]:
#@title -----------> Installation of Streamlit and pyngrok of course!!
!pip -q install streamlit
!pip -q install pyngrok

In [ ]:
#Keras pretrained model
!wget https://github.com/napoles-uach/streamlit_apps/raw/main/Streamlit_Colab/Models/brain_tumor_classification.h5

The following is the `app.py` base code. It can be modified in the cell or in the folder to the left.

In [4]:
%%writefile app.py
import streamlit as st
from img_classification import teachable_machine_classification
import keras
from PIL import Image, ImageOps
import numpy as np
st.title("Image Classification with Google's Teachable Machine")
st.header("Brain Tumor MRI Classification Example")
st.text("Upload a brain MRI Image for image classification as tumor or no-tumor")

uploaded_file = st.file_uploader("Choose a brain MRI ...", type="jpg")
if uploaded_file is not None:
  image = Image.open(uploaded_file)
  st.image(image, caption='Uploaded MRI.', use_column_width=True)
  st.write("")
  st.write("Classifying...")
  label = teachable_machine_classification(image, 'brain_tumor_classification.h5') # Name of the model from Teachablemachine
  if label == 0:
    st.write("The MRI scan has a brain tumor")
  else:
    st.write("The MRI scan is healthy")

Writing app.py


In [5]:
%%writefile img_classification.py
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np


def teachable_machine_classification(img, weights_file):
    # Load the model
    model = tensorflow.keras.models.load_model(weights_file)

    # Create the array of the right shape to feed into the keras model
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    image = img
    #image sizing
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)

    #turn the image into a numpy array
    image_array = np.asarray(image)
    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

    # Load the image into the array
    data[0] = normalized_image_array  # (Not sure if this is needed, but gives an error!!!)

    # run the inference
    prediction = model.predict(data)
    return np.argmax(prediction) # return position of the highest probability

Writing img_classification.py


In [ ]:
#@title This last cell would keep the app running. If stoped, the app would be disconnected.
from pyngrok import ngrok
public_url = ngrok.connect(port='80')
print('Link to web app:')
print (public_url)
!streamlit run --server.port 80 app.py >/dev/null